In [1]:
import numpy as np
import anndata as ad
import scanpy as sc
from scipy.sparse import csr_matrix, issparse

from sklearn.linear_model import LogisticRegression

In [2]:
adata = sc.read("adata_normalized_custom.h5ad")
adata.X = adata.layers["binarized"]

In [3]:
if issparse(adata.X):
    if not adata.X.has_sorted_indices:
        adata.X.sort_indices()

This could really use some optimization: (maybe parallelization) (took me 72min to run)

In [25]:
X_res = np.zeros(adata.X.shape)

for i in range(adata.X.shape[1]):
    y = np.asarray(np.squeeze(adata.X[:,i].toarray()))
    model = LogisticRegression(solver='newton-cholesky', warm_start = True).fit(X = X,y = y)
    X_res[:,i] = y - model.predict_proba(X)[:,1]
    if 0 == (i+1) % 500:
        print(f'Progress: {i+1} out of {adata.X.shape[1]}')


Progress: 500 out of 92653
Progress: 1000 out of 92653
Progress: 1500 out of 92653
Progress: 2000 out of 92653
Progress: 2500 out of 92653
Progress: 3000 out of 92653
Progress: 3500 out of 92653
Progress: 4000 out of 92653
Progress: 4500 out of 92653
Progress: 5000 out of 92653
Progress: 5500 out of 92653
Progress: 6000 out of 92653
Progress: 6500 out of 92653
Progress: 7000 out of 92653
Progress: 7500 out of 92653
Progress: 8000 out of 92653
Progress: 8500 out of 92653
Progress: 9000 out of 92653
Progress: 9500 out of 92653
Progress: 10000 out of 92653
Progress: 10500 out of 92653
Progress: 11000 out of 92653
Progress: 11500 out of 92653
Progress: 12000 out of 92653
Progress: 12500 out of 92653
Progress: 13000 out of 92653
Progress: 13500 out of 92653
Progress: 14000 out of 92653
Progress: 14500 out of 92653
Progress: 15000 out of 92653
Progress: 15500 out of 92653
Progress: 16000 out of 92653
Progress: 16500 out of 92653
Progress: 17000 out of 92653
Progress: 17500 out of 92653
Progr

In [26]:
adata.layers["binary_residual"] = X_res

In [27]:
adata.write("adata_custom_v2.h5ad")

In [24]:
adata.X.shape[1]

92653

In [10]:
np.asarray(np.squeeze(adata.X[:,i].toarray())).shape


(3683,)

In [15]:
model.predict_proba(X)[:,1].shape

(3683,)

In [18]:
X_res[:,3000].shape

(3683,)

In [8]:
X_res[3000:3004,50000:50009]

array([[0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [12]:
X[3000:3004]

array([[ 5563.],
       [ 2238.],
       [14162.],
       [ 9509.]])

In [ ]:
X = np.asarray(np.sum(adata.X, axis = 0)).astype(float)
X_T = X.T
X_T.shape

(92653, 1)

In [ ]:
i = 1
y = np.asarray(np.squeeze(adata.X[i,:].toarray()))
y.shape

(92653,)

In [ ]:
np.asarray(np.squeeze(adata.X[:,1].toarray())).shape

(3683,)

In [ ]:
X_res_T = np.zeros((adata.X).T.shape)

for i in range(adata.X.shape[1]):
    y = np.asarray(np.squeeze(adata.X[:,i].toarray()))
    model = LogisticRegression(solver='newton-cholesky', warm_start = True).fit(X = X_T,y = y)
    X_res_T[:,i] = y - model.predict_proba(X_T)[:,1]
    if 0 == (i+1) % 500:
        print(f'Progress: {i+1} out of {X_res_T[:,1].shape}')


Progress: 500 out of (92653,)
Progress: 1000 out of (92653,)
Progress: 1500 out of (92653,)
Progress: 2000 out of (92653,)
Progress: 2500 out of (92653,)
Progress: 3000 out of (92653,)
Progress: 3500 out of (92653,)


IndexError: row index (3683) out of range

In [ ]:
adata.layers["binary_residual_for_cells"] = csr_matrix(X_res)
adata.layers["binary_residual_for_reads"] = csr_matrix(X_res_T)

In [ ]:
adata.write("adata_custom_v2.h5ad")